In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
deathWoman = np.array(pd.read_excel('../../data/여성사망률추이.xlsx').set_index(['연도']).iloc[:43,:100])
deathMan = np.array(pd.read_excel('../../data/남성사망률추이.xlsx').set_index(['연도']).iloc[:43,:100])
dataOriDf = pd.read_excel('데이터정리3.xlsx')
ageDataOriDf = pd.read_excel('../../data/의사연령별분포.xlsx')
npAgeData = np.array(pd.read_excel('../../data/의사연령별분포.xlsx'))

In [32]:
ageDataOriDf.iloc[:,1:]

,총/합계,총/~29,총/30~39,총/40~49,총/50~59,총/60~,남/합계,남/~29,남/30~39,남/40~49,남/50~59,남/60~,여/합계,여/~29,여/30~39,여/40~49,여/50~59,여/60~,총비율/합계,총비율/~29,총비율/30~39,총비율/40~49,총비율/50~59,총비율/60~,남비율/합계,남비율/~29,남비율/30~39,남비율/40~49,남비율/50~59,남비율/60~,여비율/합계,여비율/~29,여비율/30~39,여비율/40~49,여비율/50~59,여비율/60~
0,82685,839,23488,27826,19466,11066,64371,561,14723,22244,16881,9962,18314,278,8765,5582,2585,1104,1,0.010147,0.284066,0.336530,0.235424,0.133833,0.778509,0.006785,0.178061,0.269021,0.204160,0.120481,0.221491,0.003362,0.106005,0.067509,0.031263,0.013352
1,85073,1963,25100,27820,19381,10809,65989,1356,15836,22270,16796,9731,19084,607,9264,5550,2585,1078,1,0.023074,0.295041,0.327013,0.227816,0.127056,0.775675,0.015939,0.186146,0.261775,0.197430,0.114384,0.224325,0.007135,0.108895,0.065238,0.030386,0.012671
2,88945,3459,27682,27933,19357,10514,68330,2297,17412,22366,16775,9480,20615,1162,10270,5567,2582,1034,1,0.038889,0.311226,0.314048,0.217629,0.118208,0.768228,0.025825,0.195761,0.251459,0.188600,0.106583,0.231772,0.013064,0.115465,0.062589,0.029029,0.011625
3,91197,5078,28738,27892,19258,10231,69677,3258,18114,22374,16695,9236,21520,1820,10624,5518,2563,995,1,0.055682,0.315120,0.305843,0.211169,0.112186,0.764027,0.035725,0.198625,0.245337,0.183065,0.101275,0.235973,0.019957,0.116495,0.060506,0.028104,0.010910
4,93484,6955,29528,27928,19203,9870,71066,4328,18808,22359,16656,8915,22418,2627,10720,5569,2547,955,1,0.074398,0.315862,0.298746,0.205415,0.105580,0.760194,0.046297,0.201190,0.239175,0.178170,0.095364,0.239806,0.028101,0.114672,0.059572,0.027245,0.010216
5,96180,8795,30729,27938,19154,9564,72818,5272,19852,22420,16642,8632,23362,3523,10877,5518,2512,932,1,0.091443,0.319495,0.290476,0.199147,0.099439,0.757101,0.054814,0.206405,0.233105,0.173030,0.089748,0.242899,0.036629,0.113090,0.057372,0.026118,0.009690


In [33]:
pd.set_option('display.max_row', 1200)
pd.set_option('display.max_columns', 100)

#1977년 합격자부터 2019년 합격자
dataDf = dataOriDf[['년도','의대졸합격/남', '의대졸합격/여', '의전졸합격/남', '의전졸합격/여', '불합합격/남', '불합합격/여']]
dataDf = dataDf[27:]

npData = np.around(np.array(dataDf)[:,1:7])
npData

array([[ 932.,  155.,    0.,    0.,   94.,   17.],
       [ 963.,  154.,    0.,    0.,   70.,   12.],
       [ 953.,  156.,    0.,    0.,   95.,   15.],
       [1030.,  178.,    0.,    0.,   98.,   16.],
       [1026.,  132.,    0.,    0.,  149.,   26.],
       [1220.,  111.,    0.,    0.,   59.,    8.],
       [1187.,  150.,    0.,    0.,   61.,    6.],
       [1032.,  188.,    0.,    0.,  294.,   37.],
       [1242.,  181.,    0.,    0.,  139.,   25.],
       [1656.,  302.,    0.,    0.,   94.,   14.],
       [2241.,  304.,    0.,    0.,   46.,    8.],
       [2157.,  391.,    0.,    0.,  133.,   18.],
       [2194.,  526.,    0.,    0.,  180.,   33.],
       [1983.,  493.,    0.,    0.,  260.,   62.],
       [2195.,  535.,    0.,    0.,  187.,   47.],
       [2116.,  550.,    0.,    0.,  199.,   48.],
       [2400.,  616.,    0.,    0.,   75.,   20.],
       [2282.,  522.,    0.,    0.,   81.,   21.],
       [1602.,  354.,    0.,    0.,  692.,  158.],
       [1321.,  337.,    0.,   

# 시그모이드 변형 모델

In [34]:
# 분류모델함수 생성
# x - x값, sp - 시작점, xl - x축 조정, yl - y축 조정
def sigmoid_function(x, sp, xl, yl):
    
    
    
    result = 1 / ( yl+np.exp((x-(sp+(5/xl)))*xl) )
    
    for i in range(np.size(x)):
        if x[i]<sp:
           result[i]=0
        if x[i]>(sp+(10/xl)):
           result[i]=0
            
    return result

# 분류모델 확률질량함수 생성
def makeSigmoidModel(sp, xl, yl):
    x = np.arange(1, 101, 1)
    y = sigmoid_function(x, sp, xl, yl)
    sum = np.sum(y)
    z = y/sum
    
    return z



def clusterAgeModel(rate,st,end):
    npData = np.zeros(100)
    
    for i in range(st,end):
        npData[i] = (rate**(i-end))-1  
        
    sumValue = np.sum(npData)
    npRateData = npData / sumValue
    return npRateData

In [35]:
%time clusterAgeModel(0.7,10,40)

CPU times: user 71 µs, sys: 36 µs, total: 107 µs
Wall time: 122 µs


array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.00060870e-01, 2.10040580e-01,
       1.47026377e-01, 1.02916435e-01, 7.20394754e-02, 5.04256038e-02,
       3.52958937e-02, 2.47050966e-02, 1.72915386e-02, 1.21020480e-02,
       8.46940462e-03, 5.92655423e-03, 4.14655897e-03, 2.90056228e-03,
       2.02836460e-03, 1.41782622e-03, 9.90449356e-04, 6.91285551e-04,
       4.81870888e-04, 3.35280624e-04, 2.32667439e-04, 1.60838209e-04,
       1.10557748e-04, 7.53614259e-05, 5.07240002e-05, 3.34778022e-05,
       2.14054636e-05, 1.29548266e-05, 7.03938064e-06, 2.89856850e-06,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

# 사망률 적용

In [36]:
def makeAlivePerson(npData,year):
    
    deathRate = np.array([deathMan[year], deathWoman[year]])
    resultData = npData - np.around((npData*deathRate))
    
    return resultData

# 은퇴율 적용

In [37]:
def makeWorkPerson(npData,tuningSet):
    valueList = np.zeros([2,100])
    c1 = tuningSet[2][0]/((tuningSet[0][0]**(100-tuningSet[1][0]))-1)
    c2 = tuningSet[2][1]/((tuningSet[0][1]**(100-tuningSet[1][1]))-1)
    for i in range(100):
        result1 = ((tuningSet[0][0]**(i-tuningSet[1][0]))-1)*c1
        result2 = ((tuningSet[0][1]**(i-tuningSet[1][1]))-1)*c2
        valueList[0][i] = 0 if result1<0 else 1 if result1>1 else  result1
        valueList[1][i] = 0 if result2<0 else 1 if result2>1 else  result2

    result = npData - np.around(npData*valueList)
    
    return result

In [38]:
def makeWorkPerson1(tuningSet):
    valueList = np.zeros([2,100])
    c1 = 1/((tuningSet[0][0]**(100-tuningSet[1][0]))-1)
    c2 = 1/((tuningSet[0][1]**(100-tuningSet[1][1]))-1)
    for i in range(100):
        result1 = ((tuningSet[0][0]**(i-tuningSet[1][0]))-1)*c1
        result2 = ((tuningSet[0][1]**(i-tuningSet[1][1]))-1)*c2
        valueList[0][i] = 0 if result1<0 else 1 if result1>1 else  result1
        valueList[1][i] = 0 if result2<0 else 1 if result2>1 else  result2

    
    return valueList

pd.DataFrame(makeWorkPerson1([[1.5,1.5],[25,25]]))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.105457e-14,7.763643e-14,1.475092e-13,2.523184e-13,4.095322e-13,6.453529e-13,9.990839e-13,1.529680e-12,2.325575e-12,3.519417e-12,5.310181e-12,7.996325e-12,1.202554e-11,1.806937e-11,2.713511e-11,4.073372e-11,6.113163e-11,9.172850e-11,1.376238e-10,2.064668e-10,3.097312e-10,4.646278e-10,6.969728e-10,1.045490e-09,1.568266e-09,2.352431e-09,3.528677e-09,5.293047e-09,7.939601e-09,1.190943e-08,1.786418e-08,2.679630e-08,4.019448e-08,6.029176e-08,9.043766e-08,1.356565e-07,2.034848e-07,3.052273e-07,4.578409e-07,6.867614e-07,0.000001,0.000002,0.000002,0.000003,0.000005,0.000008,0.000012,0.000018,0.000026,0.00004,0.000059,0.000089,0.000134,0.0002,0.000301,0.000451,0.000677,0.001015,0.001522,0.002284,0.003425,0.005138,0.007707,0.011561,0.017342,0.026012,0.039018,0.058528,0.087791,0.131687,0.197531,0.296296,0.444444,0.666667
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.105457e-14,7.763643e-14,1.475092e-13,2.523184e-13,4.095322e-13,6.453529e-13,9.990839e-13,1.529680e-12,2.325575e-12,3.519417e-12,5.310181e-12,7.996325e-12,1.202554e-11,1.806937e-11,2.713511e-11,4.073372e-11,6.113163e-11,9.172850e-11,1.376238e-10,2.064668e-10,3.097312e-10,4.646278e-10,6.969728e-10,1.045490e-09,1.568266e-09,2.352431e-09,3.528677e-09,5.293047e-09,7.939601e-09,1.190943e-08,1.786418e-08,2.679630e-08,4.019448e-08,6.029176e-08,9.043766e-08,1.356565e-07,2.034848e-07,3.052273e-07,4.578409e-07,6.867614e-07,0.000001,0.000002,0.000002,0.000003,0.000005,0.000008,0.000012,0.000018,0.000026,0.00004,0.000059,0.000089,0.000134,0.0002,0.000301,0.000451,0.000677,0.001015,0.001522,0.002284,0.003425,0.005138,0.007707,0.011561,0.017342,0.026012,0.039018,0.058528,0.087791,0.131687,0.197531,0.296296,0.444444,0.666667


# 출신 연령별 분류모델

In [39]:
def makeArrayUseModel(tuningList):
    model1 = clusterAgeModel(tuningList[0][0], tuningList[1][0], tuningList[2][0]) # 의대 남
    model2 = clusterAgeModel(tuningList[0][1], tuningList[1][1], tuningList[2][1]) # 의대 여
    model3 = clusterAgeModel(tuningList[0][2], tuningList[1][2], tuningList[2][2]) # 의전원 남
    model4 = clusterAgeModel(tuningList[0][3], tuningList[1][3], tuningList[2][3]) # 의전원 여
    model5 = clusterAgeModel(tuningList[0][4], tuningList[1][4], tuningList[2][4]) # 재시험 남
    model6 = clusterAgeModel(tuningList[0][5], tuningList[1][5], tuningList[2][5]) # 재시험 여
    
    resultData =  np.array([model1, model2, model3, model4, model5, model6])
    
    return resultData

# 신규인원 생성

In [40]:
def makeNewPerson(npData, tuningSet):
    oldSize = 100
    yearSize = len(npData)
    modelSize = len(npData[0])
    
    #모델 적용 배열
    applyModelArray = np.zeros((yearSize,modelSize,oldSize))

    #신규인원 배열
    newPersonArray = np.zeros((yearSize,2,oldSize))
    
    modelAry = makeArrayUseModel(tuningSet)
    
    for i in range(yearSize):
        for j in range(modelSize):
            applyModelArray[i][j] = np.around(modelAry[j]*np.around(npData[i][j]))
            if j%2==0:
                newPersonArray[i][0] += applyModelArray[i][j]
            else:
                newPersonArray[i][1] += applyModelArray[i][j]
    
    resultData = newPersonArray
    return resultData

# 최종인원 도출

In [41]:
def shiftOld(personArray):
    dataArray = np.zeros((2,len(personArray[0])))
    dataArray[0] = np.roll(personArray[0], 1)  
    dataArray[1] = np.roll(personArray[1], 1)
    dataArray[0][0] = 0
    dataArray[1][0] = 0
    
    return dataArray

def makeResultPersonArray(newPersonArray, tuningSet):
    #누적인원 배열
    
    sizeArray = list(np.shape(newPersonArray))
    yearSize = sizeArray[0]
    resultPersonArray = np.zeros(sizeArray)

    for i in range(yearSize):
        if i>0:
            # 1살 올리기 / shiftData -> 2 x 100
            shiftData = shiftOld(resultPersonArray[i-1])
            # 사망률 적용 / aliveData -> 2 x 100
            aliveData = makeAlivePerson(shiftData, i)
            # 은퇴율 적용 / workData -> 2 x 100
            workData = makeWorkPerson(aliveData, tuningSet)
            # 최종 계산
            resultPersonArray[i] =  workData + newPersonArray[i]
        else:
            resultPersonArray[i] =  newPersonArray[i]
            
            
    return resultPersonArray

# 연령별 합산

In [42]:
def sumPeopleUseAge(npData):
    yearSize = len(npData)
    
    resultData = np.zeros([yearSize, 3, 12])
    
    for i in range(yearSize):
        for j in range(2):
            # 남/여 소계 계산
            resultData[i][j][0] = np.sum(npData[i][j])
            resultData[i][j][1] = np.sum(npData[i][j][0:30])
            resultData[i][j][2] = np.sum(npData[i][j][30:40])
            resultData[i][j][3] = np.sum(npData[i][j][40:50])
            resultData[i][j][4] = np.sum(npData[i][j][50:60])
            resultData[i][j][5] = np.sum(npData[i][j][60:])
            
            # 남/여 비율 계산
            for n in range(6):
                resultData[i][j][6+n] = resultData[i][j][n]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
        
        for j in range(6):
            # 합계 계산
            resultData[i][2][j] = resultData[i][0][j] + resultData[i][1][j]  
            # 합계 비율 계산
            resultData[i][2][6+j] = resultData[i][2][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
            # 남/여 비율 계산
            resultData[i][0][6+j] = resultData[i][0][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
            resultData[i][1][6+j] = resultData[i][1][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0


    return resultData

# 코스트 계산

In [43]:
def calculateCost(npData):   
    resultData = np.zeros([6,5,5])

    for i in range(6):
        manNumCost = npData[39-i][0][1:6] - npAgeData[5-i][8:13]
        womanNumCost = npData[39-i][1][1:6] - npAgeData[5-i][14:19]
        totalNumCost = npData[39-i][2][1:6] - npAgeData[5-i][2:7]
        
        manRateCost = npData[39-i][0][7:12] - npAgeData[5-i][26:31]
        womanRateCost = npData[39-i][1][7:12] - npAgeData[5-i][32:37]
        
        npCostArray = np.array([manNumCost, womanNumCost, totalNumCost, 
                    manRateCost, womanRateCost])

        resultData[i] = npCostArray

    return resultData

In [44]:
def getCost(npData, tuningSet):
    newPerson = makeNewPerson(npData, tuningSet[0])
    resultPerson = makeResultPersonArray(newPerson,tuningSet[1])
    # sumData -> 43 x 3 x 12
    sumData = sumPeopleUseAge(resultPerson)
    costData = calculateCost(sumData)
    
    return [sumData, costData]

In [45]:
tuningSetAgeRate = [[0.9,0.7,0.1,0.1,0.9,0.7],[26,26,28,28,27,27],[39, 39, 39, 39, 39, 39]]
tuningSetRetireRate = [[1.1, 1.1],[30, 30],[0.1, 0.1]]
tuningSet = [tuningSetAgeRate, tuningSetRetireRate]

newPerson = makeNewPerson(npData, tuningSet[0])
deathPersonList = []
retirePersonList = []

resultPersonArray = np.zeros([71,2,100])

for i in range(71):
    if i>0:
        # 1살 올리기 / shiftData -> 2 x 100
        shiftData = shiftOld(resultPersonArray[i-1])
        # 사망률 적용 / aliveData -> 2 x 100
        deathRate = np.array([deathMan[i], deathWoman[i]])
        deathPerson = np.around((shiftData*deathRate))
        deathPersonList.append(deathPerson)
        aliveData = shiftData - deathPerson
        # 은퇴율 적용 / workData -> 2 x 100
        valueList = np.zeros([2,100])
        c1 = tuningSetRetireRate[2][0]/((tuningSetRetireRate[0][0]**(100-tuningSetRetireRate[1][0]))-1)
        c2 = tuningSetRetireRate[2][1]/((tuningSetRetireRate[0][1]**(100-tuningSetRetireRate[1][1]))-1)
        
        for j in range(100):
            result1 = ((tuningSetRetireRate[0][0]**(j-tuningSetRetireRate[1][0]))-1)*c1
            result2 = ((tuningSetRetireRate[0][1]**(j-tuningSetRetireRate[1][1]))-1)*c2
            valueList[0][j] = 0 if result1<0 else 1 if result1>1 else  result1
            valueList[1][j] = 0 if result2<0 else 1 if result2>1 else  result2
            
        retirePerson = np.around(aliveData*valueList)
        retirePersonList.append(retirePerson)
        workData = aliveData - retirePerson
        
        # 최종 계산
        resultPersonArray[i] =  workData + newPerson[i]
        
    else:
        resultPersonArray[i] =  newPerson[i]
            


IndexError: index 43 is out of bounds for axis 0 with size 43

In [ ]:
resultPersonArray[70]

In [ ]:
sexList = []
yearList = []
sumList = []
dfDPResult = pd.DataFrame([])

for i in range(42):
    for j in range(2):
        dfDPResult = pd.concat([dfDPResult,pd.DataFrame(deathPersonList[i][j]).T])
        sumList.append(np.sum(np.array(dfDPResult[(i*2)+j])))
        yearList.append(i+1977)
        if j%2==0:
            sexList.append('남성')
        else:
            sexList.append('여성')

# dfResult.columns = ['합계', '0~29', '30~39', '40~49', '50~59', '60~', '합계/비율', '0~29/비율','30~39/비율', '40~49/비율', '50~59/비율', '60~']
dfDPResult.insert(0,'성별',sexList,True)
dfDPResult.insert(0,'연도',yearList,True)
dfDPResult.insert(0,'합계',sumList,True)
dfDPResult.groupby(['연도', '성별']).sum()

In [ ]:
## npData = np.around(np.array(dataDf)[:,1:7])
npData
#0.9, 0.6, 1.1, 0.2
tuningSetAgeRate = [[0.9,0.6,0.9,0.6,0.9,0.6],[26,26,28,28,27,27],[39, 39, 39, 39, 39, 39]]
tuningSetRetireRate = [[1.1, 1.1],[30, 30],[0.2, 0.2]]
tuningSet = [tuningSetAgeRate, tuningSetRetireRate]
print(tuningSet)
%time data = getCost(npData,tuningSet)

In [ ]:
dfResultCost = pd.DataFrame(data[1][0])
# dfResultCost.columns = ["남성오차", "여성오차", "남여총계오차","남비율오차", "여비율오차", "비율총계오차"]
# dfResultCost.index = ["2011", "2012", "2013","2014", "2015", "2016", "합계"]
dfResultCost

In [ ]:
dfResult = pd.DataFrame(data[0][0])
indexList = []
indexList.append('1977 남성')
indexList.append('1977 여성')
indexList.append('1977 합계')
    
for i in range(42):
    dfResult = pd.concat([dfResult,pd.DataFrame(data[0][i+1])])
    indexList.append(str(i+1978)+' 남성')
    indexList.append(str(i+1978)+' 여성')
    indexList.append(str(i+1978)+' 합계')

dfResult.columns = ['합계', '0~29', '30~39', '40~49', '50~59', '60~', '합계/비율', '0~29/비율','30~39/비율', '40~49/비율', '50~59/비율', '60~']
dfResult.index = indexList
dfResult

In [ ]:
npAgeData = np.array(pd.read_excel('../../data/의사연령별분포.xlsx'))
pd.DataFrame(npAgeData).set_index(0)

In [ ]:
def test():
#     h=0.5
#     i=0.5
#     j=0.5
#     k=0.5
#     m=0.5
#     n=0.5
    
    npData = np.around(np.array(dataDf)[:,1:7])
    npData

    meanCost = np.array([])
    meanParameter = np.array([])
    resultData = []

    for h in np.arange(0.1,1,0.1):
        print("h "+ str(h))
        for i in np.arange(0.1,1,0.1):
            print("i "+ str(i))
            for j in np.arange(1,2,0.1):
                for k in np.arange(0.1,1,0.1):
                    for m in np.arange(0.1,1,0.1):
                        tuningSetAgeRate = [[h,i,j,k,h,i],[26,26,28,28,27,27],[39, 39, 39, 39, 39, 39]]
                        tuningSetRetireRate = [[1.1,1.1],[30, 30],[m, m]]
                        tuningSet = [tuningSetAgeRate, tuningSetRetireRate]
                        data = getCost(npData,tuningSet)
                        resultData.append([[h,i,j,k,m] ,data[1]])
                        
    return resultData

In [ ]:
%time saveData = test()

In [ ]:
len(saveData)

In [ ]:
resultData[0][0]

In [ ]:
dfResultCost = pd.DataFrame(saveData[0][1][0])
dfResultCost.columns = ["20대", "30대", "40대","50대", "60~"]
dfResultCost.index = ["남", "여", "남여","남비율", "여비율"]
dfResultCost

In [ ]:
cost = 0
cost += np.sum(np.abs(saveData[0][1][0][3:5,:]))
cost += np.sum(np.abs(saveData[0][1][1][3:5,:]))
cost += np.sum(np.abs(saveData[0][1][2][3:5,:]))
cost += np.sum(np.abs(saveData[0][1][3][3:5,:]))
cost += np.sum(np.abs(saveData[0][1][4][3:5,:]))
cost += np.sum(np.abs(saveData[0][1][5][3:5,:]))
cost

In [ ]:
npData = np.around(np.array(dataDf)[:,1:7])
npData

meanCost = np.array([])
meanParameter = np.array([])

In [ ]:
range(len(resultData))

In [ ]:
minCost = [1000000,1000000,[]]
for i in range(7290):
    # 비율 비교
    cost1 = 0
    cost1 += np.sum(np.abs(saveData[i][1][0][3:5,:]))
    cost1 += np.sum(np.abs(saveData[i][1][1][3:5,:]))
    cost1 += np.sum(np.abs(saveData[i][1][2][3:5,:]))
    cost1 += np.sum(np.abs(saveData[i][1][3][3:5,:]))
    cost1 += np.sum(np.abs(saveData[i][1][4][3:5,:]))
    cost1 += np.sum(np.abs(saveData[i][1][5][3:5,:]))

    # 수치비교
    cost2 = 0
    cost2 += np.sum(np.abs(saveData[i][1][0][2,:]))
    cost2 += np.sum(np.abs(saveData[i][1][1][2,:]))
    cost2 += np.sum(np.abs(saveData[i][1][2][2,:]))
    cost2 += np.sum(np.abs(saveData[i][1][3][2,:]))
    cost2 += np.sum(np.abs(saveData[i][1][4][2,:]))
    cost2 += np.sum(np.abs(saveData[i][1][5][2,:]))


    if cost1<minCost[0]:
        if cost2<minCost[1]:
            minCost = [cost1,cost2,saveData[i][0]]
        
minCost

[66763.0, [0.9, 0.30000000000000004, 1.1, 0.7000000000000001]]
[66763.0, [0.9, 0.30000000000000004, 1.1, 1.1]]
[70727.0, [0.9, 0.9, 1.1, 0.9]]

[65443.0, [0.9, 0.9, 1.1, 0.30000000000000004]]
[65211.0, [0.9, 0.1, 1.1, 0.7999999999999999]]